In [ ]:
import os
import numpy as np
import gc 

import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, io
from cellpose.io import imread

# Suite2p trace extraction
from suite2p.extraction.extract import extraction_wrapper

from cellpose_track2p.io import tiff_loader, get_session_s2p_paths
from cellpose_track2p.compute import get_spks, generate_all_meantiff, masks_to_stat, str_gad
from cellpose_track2p.plot import plot_fov_masks, plot_raster

# prevent having to refresh
%load_ext autoreload
%autoreload


In [ ]:
#TODO: add the 'skew' and maybe other fields to the stat (for compatibility with suite2p)

In [ ]:
subject_id = 'jm040'

model_type = 'cyto3'

diameter = 6
flow_threshold = 0.6
cellprob_threshold = -6.0

take_first_nsessions = None # change to None later on the server

In [ ]:
cellpose_subject_id = subject_id + '_gad'

In [ ]:
def str_degad(string):
    # remove 'gad' from string
    return string.replace('_gad', '')

In [ ]:
session_paths, all_s2p_path = get_session_s2p_paths(subject_id, take_first_nsessions=take_first_nsessions)

In [ ]:

stat = np.load(os.path.join(all_s2p_path[0], 'stat.npy'), allow_pickle=True)
redcell = np.load(os.path.join(all_s2p_path[0], 'redcell.npy'), allow_pickle=True)
iscell = np.load(os.path.join(all_s2p_path[0], 'iscell.npy'), allow_pickle=True)

In [ ]:
# make the full file structure for cellpose
for path in all_s2p_path:
    path_gad = str_gad(path, subject_id)
    if not os.path.exists(path_gad):
        os.makedirs(path_gad)
        

In [ ]:
generate_all_meantiff(all_s2p_path)

In [ ]:
img_paths = [os.path.join(str_gad(s2p_path, subject_id), 'meanImg_chan2.tiff') for s2p_path in all_s2p_path]

In [ ]:
io.logger_setup()

# model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
model = models.CellposeModel(model_type=model_type)

files = img_paths

imgs = [imread(f) for f in files]
nimg = len(imgs)

channels = [[0,0]]

masks, flows, styles = model.eval(imgs, diameter=diameter, channels=channels, flow_threshold=flow_threshold, cellprob_threshold=cellprob_threshold)

In [ ]:
# do same as above but with matching histograms of the imshow and then in a single figure (subplots) (TODO: make bottom one into a function!, this one is the same as bottom, just without the contours)

plot_fov_masks(imgs, masks, show_masks=False)
plot_fov_masks(imgs, masks)


In [ ]:
# for each one in session replace jm038 with jm038_cellpose
session_paths_gad = [str_gad(f, subject_id) for f in session_paths]

In [ ]:
masks_to_stat(masks, session_paths, session_paths_gad)

In [ ]:
def sort_tiff_paths(paths):
    # sort the paths based on the value between 'file' and '_' in the filename
    return sorted(paths, key=lambda x: int(x.split('file')[1].split('_')[0]))

In [ ]:

#  paths to motion corrected files (for now just root of session)
for session_path in session_paths_gad:
    print('starting extraction for session: ', session_path)
    ops = np.load(os.path.join(session_path, 'suite2p', 'plane0', 'ops.npy'), allow_pickle=True).item()
    stat = np.load(os.path.join(session_path, 'suite2p', 'plane0', 'stat.npy'), allow_pickle=True)
    
    tiffs_path_ch0 = os.path.join(str_degad(session_path), 'suite2p', 'plane0', 'reg_tif') # change this before deploying properly
    tiffs_path_ch1 = os.path.join(str_degad(session_path), 'suite2p', 'plane0', 'reg_tif_chan2') # change this before deploying properly

    # get all .tif files in the folder
    tiff_files_ch0 = [f for f in os.listdir(tiffs_path_ch0) if (f.endswith('_chan0.tif') and not f.startswith('.'))]
    tiff_files_ch1 = [f for f in os.listdir(tiffs_path_ch1) if (f.endswith('_chan1.tif') and not f.startswith('.'))]
    
    tiff_files_ch0 = sort_tiff_paths(tiff_files_ch0)
    tiff_files_ch1 = sort_tiff_paths(tiff_files_ch1)
    
    print('found tiff files: ', tiff_files_ch0)
    print('found tiff files: ', tiff_files_ch1)
    
    # NOTE: the loader is very memory inefficient since it loads each tiff 2x (because of page reading issue) and also it loads the whole movie for both channels at the same time - can be improved...
    tf_ch0 = tiff_loader(tiffs_path_ch0, tiff_files_ch0)
    tf_ch1 = tiff_loader(tiffs_path_ch1, tiff_files_ch1)

    # now extract the traces
    ops['allow_overlap'] = True # to avoid suite2p bug -> anyways overlap is close to 0 in sparse labelling
    stat, F, Fneu, F_chan2, Fneu_chan2 = extraction_wrapper(stat, tf_ch0, f_reg_chan2=tf_ch1, cell_masks=None, neuropil_masks=None, ops=ops)

    spks = get_spks(F, Fneu, ops)

    # plot F raster
    plot_raster(F)


    # save the traces
    F_path = os.path.join(session_path, 'suite2p', 'plane0', 'F.npy')
    Fneu_path = os.path.join(session_path, 'suite2p', 'plane0', 'Fneu.npy')
    F_chan2_path = os.path.join(session_path, 'suite2p', 'plane0', 'F_chan2.npy')
    Fneu_chan2_path = os.path.join(session_path, 'suite2p', 'plane0', 'Fneu_chan2.npy')
    spks_path = os.path.join(session_path, 'suite2p', 'plane0', 'spks.npy')
    
    np.save(F_path, F)
    np.save(Fneu_path, Fneu)
    np.save(F_chan2_path, F_chan2)
    np.save(Fneu_chan2_path, Fneu_chan2)
    np.save(spks_path, spks)
    
    # clear memory 
    del tf_ch0, tf_ch1, F, Fneu, F_chan2, Fneu_chan2, spks, ops, stat
    print('running garbage collection')
    gc.collect()
    
    
    
    
